Kody pozwalające analizę wyników wyborów do Sejmu RP z 2023 na poziomie dzielnic miasta Krakowa.

In [18]:
from collections import defaultdict, OrderedDict
import os

In [2]:
from aipolit.sejmvote.voting_factory_general_results import create_voting_general_results
from aipolit.sejmvote.voting_factory_candidate_results import create_voting_candidate_results
from aipolit.utils.text import save_tsv

In [3]:
ELECTIONS_ID = 'sejm2023'
OKREG_NO = 13
PROCESSED_CITY = 'Kraków'  # we really filter by "powiat name"!!!
MIN_OBWOD_POPULATION = 200 # we exclude "small" komisje to exclude hospitals etc., which may skew the results

In [15]:
ROOT_OUT_DIR = os.path.join(os.getenv("HOME"), "Pulpit")

In [4]:
general_results_data = create_voting_general_results(elections_id=ELECTIONS_ID)
candidate_results_data = create_voting_candidate_results(elections_id=ELECTIONS_ID, okreg_no=OKREG_NO)

In [5]:
obwod_ids_from_city = \
    general_results_data.voting_place_data.get_obwod_ids_matching_criteria(
        powiat_name=PROCESSED_CITY,
        with_location_data=True,
        min_population=MIN_OBWOD_POPULATION,
    )

print(f"We have {len(obwod_ids_from_city)} obwods to check in {PROCESSED_CITY}")

We have 403 obwods to check in Kraków


Krok 1: Ręczne przyporządkowanie obwodów
============================

Uznałem, że tak będzie zwyczajnie szybciej i łatwiej :D

Spojrzałem na wykaz i ręcznie przepisałem dzielnice:
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/415332/karta


In [6]:
def dzielnica_no_to_name(obwod_no:int) -> str:
    names = {
        1: "01 - Stare Miasto",
        2: "02 - Grzegórzki",
        3: "03 - Prądnik Czerwony",
        4: "04 - Prądnik Biały",
        5: "05 - Krowodrza",
        6: "06 - Bronowice",
        7: "07 - Zwierzyniec",
        8: "08 - Dębniki",
        9: "09 - Łagiewniki-Borek Fałęcki",
        10: "10 - Swoszowice",
        11: "11 - Podgórze Duchackie",
        12: "12 - Bieżanów Prokocim",
        13: "13 - Podgórze",
        14: "14 - Czyżyny",
        15: "15 - Mistrzejowice",
        16: "16 - Bieńczyce",
        17: "17 - Wzgórza Krzesławickie",
        18: "18 - Nowa Huta"
    }
    return names.get(obwod_no, None)

In [7]:
def obdwod_no_to_dzielnica_no(obwod_no: int) -> int:
    if obwod_no >= 1 and obwod_no <= 23:
        return 1 # Stare Miasto
    elif obwod_no >= 24 and obwod_no <= 41:
        return 2 # Grzegórzki
    elif obwod_no >= 42 and obwod_no <= 68:
        return 3 # Prądnik Czerwony
    elif obwod_no >= 69 and obwod_no <= 105:
        return 4 # Prądnik Biały
    elif obwod_no >= 106 and obwod_no <= 124:
        return 5 # Krowodrza
    elif obwod_no >= 125 and obwod_no <= 137:   
        return 6 # Bronowice
    elif obwod_no >= 138 and obwod_no <= 148:
        return 7 # Zwierzyniec
    elif obwod_no >= 149 and obwod_no <= 184:
        return 8 # Dębniki
    elif obwod_no >= 185 and obwod_no <= 192:
        return 9 # Łagiewniki Borek Fałęcki
    elif obwod_no >= 193 and obwod_no <= 208:
        return 10 # Swoszowice
    elif obwod_no >= 209 and obwod_no <= 236:
        return 11 # Podgórze Duchackie
    elif obwod_no >= 237 and obwod_no <= 270:
        return 12 # Bieżanów Prokocim
    elif obwod_no >= 271 and obwod_no <= 292:
        return 13 # Podgórze
    elif obwod_no >= 293 and obwod_no <= 309:
        return 14 # Czyżyny
    elif obwod_no >= 310 and obwod_no <= 335:
        return 15 # Mistrzejowice
    elif obwod_no >= 336 and obwod_no <= 359:
        return 16 # Bieńczyce
    elif obwod_no >= 360 and obwod_no <= 370:
        return 17 # Wzgorza Krzesławickie
    elif obwod_no >= 371 and obwod_no <= 405:
        return 18 # Nowa Huta
    
    # wyjątki
    elif obwod_no == 406:
        return 8
    elif obwod_no == 407:
        return 8
    elif obwod_no == 408:
        return 12
    elif obwod_no == 409:
        return 6
    elif obwod_no == 410:
        return 14
    elif obwod_no == 411:
        return 17
    
    return None

In [8]:
processed_obwod_numbers = []

obwod_no_to_obwod_id = {}
obwod_id_to_obwod_no= {}
dzielnica_no_to_obwod_nos = defaultdict(set)
dzielnica_no_to_obwod_ids = defaultdict(set)

for obwod_id in obwod_ids_from_city:
    (_, obwod_no) = obwod_id.split("===")
    obwod_no = int(obwod_no)
    
    dzielnica_no = obdwod_no_to_dzielnica_no(obwod_no)
    if dzielnica_no is None:
        continue
        
    dzielnica_no_to_obwod_nos[dzielnica_no].add(obwod_no)
    dzielnica_no_to_obwod_ids[dzielnica_no].add(obwod_id)
        
    obwod_no_to_obwod_id[obwod_no] = obwod_id
    obwod_id_to_obwod_no[obwod_id] = obwod_no
    
    processed_obwod_numbers.append(obwod_no)
    
print(f"We have {len(processed_obwod_numbers)} processed_obwod numbers")

We have 403 processed_obwod numbers


Obliczenia dla kandydata
=========================


In [48]:
def count_for_lista_for_given_obwod_id(label, lista_id, obwod_ids):
    stats = {
        'name': label,
        'population': 0,
        'valid_votes': 0,
        'votes_on_list': 0,
    }
    for obwod_id in obwod_ids:
        entry = general_results_data.get_results_entry_by_obwod_id(obwod_id)
        stats['population'] += entry['total_possible_voters']
        stats['valid_votes'] += entry['total_valid_votes']
        stats['votes_on_list'] += entry[f"total_votes_lista_{lista_id}"]
    return stats

def add_perc(results, key, total_key, perc_key=None):
    if perc_key is None:
        perc_key = key + "_perc"
    
    for dzielnica_no in results.keys():
        val = results[dzielnica_no][key]
        total = results[dzielnica_no][total_key]

        perc_val = int(1000000 * val / total) / 10000
        results[dzielnica_no][perc_key] = perc_val
        
def add_ranking(results, key):
    rank_key = key + "_rank"

    current_rank = 0
    for dzielnica_no in sorted(results.keys(), key=lambda d: -results[d][key]):
        if dzielnica_no == 0:
            results[dzielnica_no][rank_key] = 0    
        else:
            current_rank += 1
            results[dzielnica_no][rank_key] = current_rank
    

def count_for_lista(lista_id):
    results = OrderedDict()
    # dla wszystkich
    results[0] = count_for_lista_for_given_obwod_id("wszystkie", lista_id, obwod_ids_from_city)
    
    # per dzielinca
    for dzielnica_no in range(1, 19):
        obwod_ids_from_dzielnica = dzielnica_no_to_obwod_ids[dzielnica_no]
        results[dzielnica_no] = \
            count_for_lista_for_given_obwod_id(
                dzielnica_no_to_name(dzielnica_no), 
                lista_id,
                obwod_ids_from_dzielnica)
        
    add_perc(results, "valid_votes", "population")
    add_perc(results, "votes_on_list", 'valid_votes')
    add_ranking(results, "votes_on_list_perc")
        
    return results

In [28]:
results_for_lista = count_for_lista('3d')

In [29]:
save_tsv(
         os.path.join(ROOT_OUT_DIR, "lista_results.tsv"),
         results_for_lista.values(),
         header=['name', 'population', 'valid_votes','valid_votes_perc', 'votes_on_list', 'votes_on_list_perc', 'votes_on_list_perc_rank'],
         labels=['Nazwa dzielnicy', 'Populacja', 'Oddanych głosów', 'Frekwencja', 'Głosów na 3D', 'Głosów na 3D (w %)', 'Ranking 3D']
        )

In [49]:
def count_for_candidate_for_given_obwod_id(dzielnica_no, label, lista_id, cand_index, obwod_ids):
    stats = results_for_lista[dzielnica_no].copy()
    stats['my_votes'] = 0
    for obwod_id in obwod_ids:
        entry = candidate_results_data.get_results_entry_by_obwod_id(obwod_id)
        my_votes = entry[f"total_votes_lista_{lista_id}_cand_{cand_index}"]
        stats['my_votes'] += my_votes
    return stats

def count_for_candidate(lista_id, cand_index):
    results = OrderedDict()
    # dla wszystkich dzielnic
    results[0] = count_for_candidate_for_given_obwod_id(0, "wszystkie", lista_id, cand_index, obwod_ids_from_city)
    
    # per dzielinca
    for dzielnica_no in range(1, 19):
        obwod_ids_from_dzielnica = dzielnica_no_to_obwod_ids[dzielnica_no]
        results[dzielnica_no] = \
            count_for_candidate_for_given_obwod_id(
                dzielnica_no,
                dzielnica_no_to_name(dzielnica_no), 
                lista_id,
                cand_index,
                obwod_ids_from_dzielnica)
    
    add_perc(results, "my_votes", 'valid_votes')
    add_perc(results, "my_votes", 'votes_on_list', 'my_votes_perc_from_list')
    add_ranking(results, "my_votes_perc")
    add_ranking(results, "my_votes_perc_from_list")
    
    return results

In [50]:
cands_to_process = [
    (0, 'Rafał Komarewicz'),
    (2, 'Małgorzata Szostak'),
    (4, 'Karolina Górnisiewicz'),
    (6, 'Monika Motyczyńska'),
    (8, 'Patrycja Durska'),
    (12, 'Jacek Stawowski'),
]

for cand_entry in cands_to_process:
    result_for_cand = count_for_candidate('3d', cand_entry[0])
    fp = os.path.join(ROOT_OUT_DIR, f"cand-{cand_entry[1]}.tsv")
    
    save_tsv(
         fp,
         result_for_cand.values(),
         header=['name', 'population', 'valid_votes','valid_votes_perc', 'votes_on_list', 'votes_on_list_perc', 'votes_on_list_perc_rank',
                'my_votes', 'my_votes_perc', 'my_votes_perc_rank', 'my_votes_perc_from_list', 'my_votes_perc_from_list_rank'
                ],
         labels=['Nazwa dzielnicy', 'Populacja', 'Oddanych głosów', 'Frekwencja', 'Głosów na 3D', 'Głosów na 3D (w %)', 'Ranking 3D',
                'moje głosy', 'moje głosy (w %)', 'Mój RANKING (ogólny)', 'moje głosy (w % względem głosów listy)', 'Mój RANKING - względny (mój wkład w listę)', 
                ]
        )

Wszystkie wygenerowane pliki TSV wystarczy teraz połączyć w jeden Excel (nie chciało mi się generować XLSX :) a złożenie tego ręczne zajmuje 2 minuty :D ).

W sumie tyle.